# Seattle Fleet data

* [Active Fleet Complement](https://data.seattle.gov/City-Business/Active-Fleet-Complement/enxu-fgzb)
* [Current Fleet Surplus/Auction List](https://data.seattle.gov/Finance/Current-Fleet-Surplus-Auction-List/6gnm-7jex)
* [Sold Fleet Equipment](https://data.seattle.gov/City-Business/Sold-Fleet-Equipment/y6ef-jf2w)

In [18]:
from collections import Counter
import json

import requests

In [2]:
APIS = {
    'active': 'https://data.seattle.gov/resource/enxu-fgzb.json',
    'surplus': 'https://data.seattle.gov/resource/6gnm-7jex.json',
    'sold': 'https://data.seattle.gov/resource/y6ef-jf2w.json',
}

In [3]:
def get_data(url: str) -> list[dict]:
    """Add 5k paramter limit to all requests"""
    r = requests.get(url + '?$limit=5000')
    return r.json()

## Active Fleet

In [4]:
active = get_data(APIS.get('active'))

print(len(active))
active[0]

4054


{'equipid': '30923',
 'license': '51406D',
 'year': '2010',
 'make': 'FORD',
 'model': 'F350',
 'descrip': 'PICKUP - FLATBED - CLASS 4',
 'dept': 'SDOT',
 'inservicedate': '2010-02-01T00:00:01.000',
 'estimated_replacement_year': '2020',
 'equipment_type': 'TRUCK - LIGHT',
 'inservicecost': '51593.79',
 'asset_weight_category': 'CLASS 3',
 'system_group': 'PICKUP',
 'fuel_type': 'DSL'}

In [5]:
unique_make = sorted(list(set([a['make'] for a in active])))
unique_model = sorted(list(set([a['model'] for a in active])))

total_inservice_cost = round(sum([float(a['inservicecost']) for a in active]), 2)

print(f' Unique manufacturers: {len(unique_make):,}')
print(f'Unique vehicle models: {len(unique_model):,}')
print(f' Total inservice cost: ${total_inservice_cost:,.2f}')

 Unique manufacturers: 239
Unique vehicle models: 607
 Total inservice cost: $383,356,040.64


In [33]:
print(f'Total ten deptartments by vehicle count:')
for dept, cnt in Counter([a['dept'] for a in active]).most_common(10):
    print(f'{dept:>4} -> {cnt}')

Total ten deptartments by vehicle count:
 SCL -> 915
 SPU -> 781
 SPD -> 634
 DPR -> 591
SDOT -> 548
 SFD -> 247
 FAS -> 129
SDCI -> 104
 CEN -> 28
 SPL -> 25


## Surplus

In [6]:
surplus = get_data(APIS.get('surplus'))

print(len(surplus))
surplus[0]

35


{'equip_id': '83459',
 'year': '2008',
 'make': 'FORD',
 'model': 'E350',
 'color': 'WHITE',
 'description': 'CARGO - CUTAWAY VAN',
 'asset_weight_category': 'CLASS 2',
 'system_group': 'VAN',
 'fuel_type': 'UNL',
 'auction_house': 'MURPHYS AUCTION',
 'comments': 'REPLACED',
 'vin': '1FDWE35L38DB52113',
 'department': 'SPU',
 'retirement_date': '2022-04-11T00:00:01.000',
 'equipment_type': 'VAN'}

In [17]:
Counter([s['auction_house'] for s in surplus]).most_common()

[('BIDADOO ONLINE AUCTION', 20), ('MURPHYS AUCTION', 15)]

## Sold

In [8]:
sold = get_data(APIS.get('sold'))

print(len(sold))
sold[0]

1021


{'equip_id': 'EG0115',
 'year': '1994',
 'make': 'CATERPILLAR',
 'model': '416B',
 'description': 'SOLID WASTE BACKHOE LOADER',
 'dept': 'SPU',
 'sale_price': '0.00',
 'sale_date': '2021-06-24T00:00:01.000',
 'sold_by': 'MURPHYS AUCTION',
 'vin': '8ZK03229',
 'equipment_type': 'LOADER - BACKHOE',
 'asset_weight_category': 'OFFROAD HEAVY',
 'system_group': 'CONSTRUCTION',
 'fuel_type': 'DSL'}

In [9]:
total_sold = sum([float(s['sale_price']) for s in sold])
print(f'Total sold: ${total_sold:,.2f}')

Total sold: $9,915,145.62


-------------

## overlap of data sets

In [16]:
active_ids = set([a['equipid'] for a in active])
surplus_ids = set([a['equip_id'] for a in surplus])
sold_ids = set([a['equip_id'] for a in sold])

### active & surplus

In [ ]:
# no active ids are listed as surplus
active_ids & surplus_ids

### active & sold

In [11]:
# no active ids are listed as sold
active_ids & sold_ids

set()

### surplus & sold

In [12]:
# a few surplus ids are listed as sold
recently_sold = surplus_ids & sold_ids
recently_sold

{'31969', 'G75413', 'PD3078'}

In [13]:
# reorganize to a dict of equip_id -> rest of dict
recently_sold_surplus = {s['equip_id']: s for s in surplus if s['equip_id'] in recently_sold}
recently_sold_sold = {s['equip_id']: s for s in sold if s['equip_id'] in recently_sold}

In [14]:
overlap = []
for eid in list(recently_sold):
    recent_surplus = recently_sold_surplus.get(eid)
    recent_sold = recently_sold_sold.get(eid)

    overlap.append(recent_surplus | recent_sold)  # merge operator as of 3.9

In [15]:
for o in overlap:
    print(json.dumps(o, indent=2))

{
  "equip_id": "G75413",
  "year": "2006",
  "make": "PETERBILT",
  "model": "378",
  "description": "SOLID WASTE TRUCK",
  "asset_weight_category": "CLASS 8",
  "system_group": "TRUCK",
  "fuel_type": "DSL",
  "auction_house": "BIDADOO ONLINE AUCTION",
  "comments": "REPLACED",
  "vin": "1XPFDBOX37D686142",
  "department": "SPU",
  "retirement_date": "2019-07-17T00:00:01.000",
  "equipment_type": "TRUCK - HEAVY",
  "dept": "SPU",
  "sale_price": "53100.00",
  "sale_date": "2019-07-16T00:00:01.000",
  "sold_by": "BIDADOO ONLINE AUCTION"
}
{
  "equip_id": "31969",
  "year": "2012",
  "make": "JOHN DEERE",
  "model": "GATOR",
  "description": "SOLID WASTE GATOR - ELECTRIC",
  "asset_weight_category": "OFFROAD LIGHT",
  "system_group": "ATV-UTILITY VEHICLE",
  "fuel_type": "ELE",
  "auction_house": "BIDADOO ONLINE AUCTION",
  "comments": "REPLACED",
  "vin": "TBD",
  "department": "SPU",
  "retirement_date": "2021-09-10T00:00:01.000",
  "equipment_type": "UTILITY VEHICLE - ELECTRIC",
  "